Networks - Final Project

"Is there an optimal road or road structure to close and convert to a public use space in Barcelona that minimizes the impact on traffic?"

-Clarice Mottet, Amber Walker, Mox Ballo

General Notes:

There are only 930 nodes in the link_file, there are 2522 edges in the link_file (haven't checked for duplicates). There are no duplicate edges (based on origin_node to to_node)

website link that the frank wolfe code is from

https://nbviewer.org/github/PyTrans/Urban-Network-Analysis/blob/master/Trip_Assignment-Frank-Wolfe_Algorithm.ipynb

website link that the TransportationNetworks code is from

https://github.com/PyTrans/Urban-Network-Analysis/tree/master/pytrans/UrbanNetworkAnalysis

SUPER IMPORTANT

I saw in the TransportationNetworks.py file that their initial alpha was .5 but everywhere said it should be .15, so I changed it to .15 in my file.

Because there is no node_file and I have to create it, I had to alter some of the TransportationNetworks.py code

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
#libraries

import pandas as pd
import numpy as np
import networkx as nx
import scipy.integrate as integrate 
from scipy.optimize import minimize_scalar
import matplotlib.pyplot as plt
from scipy.misc import derivative
# import TransportationNetworks as tn

# import os
# os.chdir('/content/drive/My Drive/networks_final_project_collab/networks_finalproject')

import TransportationNetworks_CM as tncm

# path_in_ = r'./inputs/'
# path_out_ = r'./outputs'
# path_out_results_ = r'./outputs/results'

path_in_ = r'/home/clarice/Documents/VSCode/Term2_Networks/final_project/networks_finalproject/inputs/'
path_out_ = r'/home/clarice/Documents/VSCode/Term2_Networks/final_project/networks_finalproject/outputs'
path_out_results_ = r'/home/clarice/Documents/VSCode/Term2_Networks/final_project/networks_finalproject/outputs/results'

In [3]:
#functions: user, based on jupyter notebook

#from website to calculate latency
def BPR(t0, xa, ca, alpha, beta):
    ta = t0*(1+alpha*(xa/ca)**beta)
    return ta

#from website: method to calculate nash equilibrium (when SO = False)
def calculateZ(theta, network, SO):
    z = 0
    for linkKey, linkVal in network.items():
        t0 = linkVal['t0']
        ca = linkVal['capa']
        beta = linkVal['beta']
        alpha = linkVal['alpha']
        aux = linkVal['auxiliary'][-1]
        flow = linkVal['flow'][-1]
        
        if SO == False:
            z += integrate.quad(lambda x: BPR(t0, x, ca, alpha, beta), 0, flow+theta*(aux-flow))[0]
        elif SO == True:
            z += list(map(lambda x : x * BPR(t0, x, ca, alpha, beta), [flow+theta*(aux-flow)]))[0]
    return z

#from website: finds nash equilibrium
def lineSearch(network, SO):
    theta = minimize_scalar(lambda x: calculateZ(x, network, SO), bounds = (0,1), method = 'Bounded')
    return theta.x

In [4]:
#functions: user, based on jupyter notebook

def initialize(link_file, trip_file, od_vols, origins, od_dic, links, graph):
    #set objective and open the network
    SO = False # True - System optimum, False - User equilibrium
    barcelonaSubset = tncm.Network(link_file, trip_file, od_vols, origins, od_dic, links, graph)

    #CM - start of initialization

    # define output variables, network and fwResult
    network = {(u,v): {'t0':d['object'].t0, 'alpha':d['object'].alpha, \
            'beta':d['object'].beta, 'capa':d['object'].capacity, 'flow':[], \
            'auxiliary':[], 'cost':[]} for (u, v, d) in barcelonaSubset.graph.edges(data=True)}

    fwResult = {'theta':[], 'z':[]}

    # initial all-or-nothing assignment and update link travel time(cost)
    barcelonaSubset.all_or_nothing_assignment()
    barcelonaSubset.update_linkcost()

    for linkKey, linkVal in network.items():
        linkVal['cost'].append(barcelonaSubset.graph[linkKey[0]][linkKey[1]]['weight'])
        linkVal['auxiliary'].append(barcelonaSubset.graph[linkKey[0]][linkKey[1]]['object'].vol)
        linkVal['flow'].append(barcelonaSubset.graph[linkKey[0]][linkKey[1]]['object'].vol)

    return barcelonaSubset, network, fwResult

#calculate nash equilibrium
def calc_nash(barcelonaSubset, network, fwResult):
    SO=False
    ## iterations
    iterNum=0
    iteration = True
    while iteration:
        iterNum += 1
        barcelonaSubset.all_or_nothing_assignment()
        barcelonaSubset.update_linkcost()
        
        # set auxiliary flow using updated link flow
        for linkKey, linkVal in network.items():
            linkVal['auxiliary'].append(barcelonaSubset.graph[linkKey[0]][linkKey[1]]['object'].vol)
            
        # getting optimal move size (theta)
        theta = lineSearch(network, SO)
        fwResult['theta'].append(theta)
        
        # set link flow (move) based on the theta, auxiliary flow, and link flow of previous iteration
        for linkKey, linkVal in network.items():
            aux = linkVal['auxiliary'][-1]
            flow = linkVal['flow'][-1]
            linkVal['flow'].append(flow + theta*(aux-flow))
            
            barcelonaSubset.graph[linkKey[0]][linkKey[1]]['object'].vol =  flow + theta * (aux - flow)
            barcelonaSubset.graph[linkKey[0]][linkKey[1]]['object'].flow = flow + theta * (aux - flow)
            
        # update link travel time
        barcelonaSubset.update_linkcost()
        
        # calculate objective function value
        z=0
        for linkKey, linkVal in network.items():
            linkVal['cost'].append(barcelonaSubset.graph[linkKey[0]][linkKey[1]]['weight'])
            totalcost = barcelonaSubset.graph[linkKey[0]][linkKey[1]]['object'].get_objective_function()
            z+=totalcost
            
        fwResult['z'].append(z)        
            
        # convergence test
        if iterNum == 1:
            iteration = True
        else:
            if abs(fwResult['z'][-2] - fwResult['z'][-1]) <= 0.001 or iterNum==3000:
                iteration = False

    return barcelonaSubset, network, fwResult

In [5]:
#functions: user - based on TransportationNetworks

def open_trip_file_CM(trip_file):
    demand_factor=1.0
    f = open(trip_file)
    lines = f.readlines()
    f.close()

    od_vols = {}
    current_origin = None

    for line in lines:
        if current_origin == None and line.startswith("Origin"):
            origin = str(int(line.split("Origin")[1]))
            current_origin = origin

        elif current_origin != None and len(line) < 3:
            # print "blank",line,
            current_origin = None

        elif current_origin != None:
            to_process = line[0:-2]
            for el in to_process.split(";"):
                try:
                    dest = str(int(el.split(":")[0]))
                    demand = float(el.split(":")[1]) * demand_factor
                    od_vols[current_origin, dest] = demand
                except:
                    continue
    origins = [str(i) for i, j in od_vols]
    origins = list(dict.fromkeys(origins).keys())

    od_dic = {}
    for (origin, destination) in od_vols:
        if origin not in od_dic:
            od_dic[origin] = {}

        od_dic[origin][destination] = od_vols[origin, destination]
    return od_vols, origins, od_dic


def open_link_file_CM(link_file):
    SO = False
    link_fields = {"from": 1, "to": 2, "capacity": 3, "length": 4, "t0": 5, \
                    "B": 6, "beta": 7, "V": 8}
    f = open(link_file)
    lines = f.readlines()
    f.close()

    links_info = []

    header_found = False
    for line in lines:
        if not header_found and line.startswith("~"):
            header_found = True
        elif header_found:
            links_info.append(line)

    nodes = {}
    links = []

    for line in links_info:
        data = line.split("\t")

        try:
            origin_node = str(int(data[link_fields["from"]]))
        except IndexError:
            continue
        to_node = str(int(data[link_fields["to"]]))
        capacity = float(data[link_fields["capacity"]])
        length = float(data[link_fields["length"]])
        alpha = float(data[link_fields["B"]])
        beta = float(data[link_fields["beta"]])

        if origin_node not in nodes:
            n = tncm.Node(node_id=origin_node)
            nodes[origin_node] = n

        if to_node not in nodes:
            n = tncm.Node(node_id=to_node)
            nodes[to_node] = n

        l = tncm.Link(link_id=len(links), length=length, capacity=capacity, alpha=alpha, beta=beta,
                    from_node=origin_node, to_node=to_node, flow=float(0.0), SO=SO)

        links.append(l)
    return links
    
def build_datastructure_CM(links):        
    graph = nx.DiGraph()

    for l in links:
        graph.add_edge(l.get_from_node(), l.get_to_node(), object=l, time=l.get_time())
    return graph

In [ ]:
#functions: user, scratch

#used in the identify superblocks code
def common_rmv_round(graph, neighbors_node_, common_node):
    for node_ in neighbors_node_:
        neighbors_node__ = list(graph.neighbors(node_))
        for node_common in common_node:
            if node_common in neighbors_node__:
                common_node.append(node_)
                common_node = list(set(list(common_node)))
                neighbors_node_.remove(node_)        
    return neighbors_node_, common_node

#function that identifies superblocks in an undirected graph
def identify_superblocks(graph):
    list_nodes = list(graph.nodes)

    dict_blocks = {}
    iter_ = 0

    for node_i in list_nodes:
        # for node_i in list_nodes:
        neighbors_i = list(graph.neighbors(node_i))
        neighbors_i_ = list(graph.neighbors(node_i))
        nodes_common = []
        for node_j in neighbors_i:
            neighbors_j = list(graph.neighbors(node_j))
            neighbors_j.remove(node_i)
            neighbors_i_.remove(node_j)
            #make sure that you remove common neighbors in neighbors_j
            for node_j_ in neighbors_i:
                if node_j != node_j_:
                    neighbors_j_ = list(graph.neighbors(node_j_))
                    for element in neighbors_j_:
                        if element in neighbors_j:
                            neighbors_j.remove(element)
                            nodes_common.append(element)
                            nodes_common = list(set(list(nodes_common)))
            #continue to next node
            for node_k in neighbors_j:
                neighbors_k = list(graph.neighbors(node_k))
                neighbors_k.remove(node_j)
                #remove common neighbors
                neighbors_k, nodes_common = common_rmv_round(graph, neighbors_k, nodes_common)
                for node_l in neighbors_k:
                    neighbors_l = list(graph.neighbors(node_l))
                    neighbors_l.remove(node_k)
                    #now we have (node_i, node_j, node_k, node_l)
                    #go to row 2
                    for node_m in neighbors_i_:
                        #we don't want node_j to equal node_m
                        neighbors_m = list(graph.neighbors(node_m))
                        neighbors_m.remove(node_i)
                        for node_n in neighbors_m:
                            neighbors_n = list(graph.neighbors(node_n))
                            neighbors_n.remove(node_m)
                            for node_o in neighbors_n:
                                if node_o != node_j:
                                    neighbors_o = list(graph.neighbors(node_o))
                                    neighbors_o.remove(node_n)
                                    for node_p in neighbors_o:
                                        if node_p != node_k:
                                            neighbors_p = list(graph.neighbors(node_p))
                                            neighbors_p.remove(node_o)
                                            #now we have (node_m, node_n, node_o, node_p)
                                            #if first row is connected to second row
                                            if (node_j in neighbors_n)&(node_k in neighbors_o)&(node_l in neighbors_p):
                                                #continue to third row
                                                for node_q in neighbors_m:
                                                    neighbors_q = list(graph.neighbors(node_q))
                                                    neighbors_q.remove(node_m)
                                                    for node_r in neighbors_q:
                                                        neighbors_r = list(graph.neighbors(node_r))
                                                        neighbors_r.remove(node_q)
                                                        for node_s in neighbors_r:
                                                            if node_s != node_n:
                                                                neighbors_s = list(graph.neighbors(node_s))
                                                                neighbors_s.remove(node_r)
                                                                for node_t in neighbors_s:
                                                                    if node_t !=node_o:
                                                                        neighbors_t = list(graph.neighbors(node_t))
                                                                        neighbors_t.remove(node_s)
                                                                        if (node_n in neighbors_r)&(node_o in neighbors_s)&(node_p in neighbors_t):
                                                                            #now we have (node_q, node_r, node_s, node_t)
                                                                            # print("third row",node_q, node_r, node_s, node_t)
                                                                            for node_w in neighbors_q:
                                                                                neighbors_w = list(graph.neighbors(node_w))
                                                                                neighbors_w.remove(node_q)
                                                                                for node_x in neighbors_w:
                                                                                    neighbors_x = list(graph.neighbors(node_x))
                                                                                    neighbors_x.remove(node_w)
                                                                                    for node_y in neighbors_x:
                                                                                        neighbors_y = list(graph.neighbors(node_y))
                                                                                        neighbors_y.remove(node_x)
                                                                                        for node_z in neighbors_y:
                                                                                            if node_z != node_s:
                                                                                                neighbors_z = list(graph.neighbors(node_z))
                                                                                                neighbors_z.remove(node_y)
                                                                                                if (node_r in neighbors_x)&(node_s in neighbors_y)&(node_t in neighbors_z):
                                                                                                    #document all the nodes                                                                
                                                                                                    list_center = [node_n, node_o, node_r, node_s]
                                                                                                    list_corner = [node_i, node_l, node_w, node_z]
                                                                                                    list_outside = [node_i, node_j, node_k, node_l, node_m, node_p, node_q, node_t, node_w, node_x, node_y, node_z]

                                                                                                    list_center.sort()
                                                                                                    list_corner.sort()
                                                                                                    list_outside.sort()

                                                                                                    #make sure no duplicate nodes    
                                                                                                    list_center = list(set(list_center))
                                                                                                    list_outside = list(set(list_outside))

                                                                                                    #check that all nodes are degree 4
                                                                                                    all_nodes_degree4 = 0
                                                                                                    for node in list_outside:
                                                                                                        if graph.degree[node] == 4:
                                                                                                            all_nodes_degree4 += 1
                                                                                                    for node in list_center:
                                                                                                        if graph.degree[node] == 4:
                                                                                                            all_nodes_degree4 += 1
                                                                                                    
                                                                                                    if all_nodes_degree4 == 16:
                                                                                                        print("all nodes are degree 4")
                                                                                                        #check to make sure that the same graph isn't being added again
                                                                                                        are_equal = 0
                                                                                                        for prev in dict_blocks.keys():
                                                                                                            if (dict_blocks[prev][0] == list_center)&(dict_blocks[prev][2] == list_outside):
                                                                                                                are_equal += 1
                                                                                                        if are_equal == 0:
                                                                                                            #add the new superblock to the dictionary
                                                                                                            dict_blocks[iter_] = [list_center, list_corner, list_outside]
                                                                                                            iter_ += 1

    return dict_blocks

In [6]:
#functions - user



#remove edges from consideration

#remove mini block edges from net file

#remove mini block trips from trips file
#(if the node is within the mini block it has to be removed 
#from the origin and destinactions trips file)

#calc total latency

#store and export results

In [7]:
#import data into programming environment
directory = path_in_
link_file = '{}Barcelona_net.tntp'.format(path_in_)
trip_file = '{}Barcelona_trips.tntp'.format(path_in_)

od_vols, origins, od_dic = open_trip_file_CM(trip_file)
links = open_link_file_CM(link_file)
graph = build_datastructure_CM(links)
#to be able to access stuff in graph

#subset to just one neighborhood of Barcelona

# barcelonaSubset = tn.Network(link_file, trip_file, SO=False)
# barcelonaSubset = tncm.Network(link_file, trip_file, od_vols, origins, od_dic, links, graph)

In [8]:
print(graph.edges())

[(1, 290), (1, 307), (1, 316), (290, 1), (290, 276), (290, 289), (307, 1), (307, 308), (307, 311), (307, 312), (316, 1), (316, 300), (316, 311), (316, 314), (2, 302), (2, 304), (302, 2), (302, 301), (302, 303), (302, 304), (304, 2), (304, 76), (304, 79), (304, 80), (304, 85), (304, 90), (304, 302), (304, 303), (304, 1020), (3, 301), (3, 306), (301, 3), (301, 4), (301, 302), (301, 306), (306, 3), (306, 4), (306, 301), (306, 308), (306, 310), (306, 1020), (4, 301), (4, 306), (5, 298), (5, 299), (5, 310), (298, 5), (298, 274), (298, 297), (299, 5), (299, 288), (299, 297), (299, 300), (310, 5), (310, 278), (310, 306), (310, 309), (6, 424), (6, 425), (6, 456), (424, 6), (424, 423), (424, 427), (424, 452), (425, 6), (425, 279), (425, 428), (456, 6), (456, 8), (456, 488), (456, 489), (7, 274), (7, 281), (7, 283), (274, 7), (274, 280), (274, 282), (274, 360), (274, 433), (281, 202), (281, 7), (281, 272), (281, 284), (283, 7), (283, 280), (283, 284), (283, 437), (8, 272), (8, 456), (8, 474), (8

# initialization

In [9]:

#initialize graph
# barcelonaSubset, network, fwResult = initialize(link_file, trip_file, od_vols, origins, od_dic, links, graph)

#find nash equilibrium
# barcelonaSubset, network, fwResult = calc_nash(barcelonaSubset, network, fwResult)

#calculate latency

#export flow
#export latency

In [15]:
print(origins)

['1', '3', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99']


In [16]:
i = 1
num = nx.single_source_dijkstra(graph, i, weight="weight")
print(num)

({1: 0, 290: 1, 307: 1, 316: 1, 276: 2, 289: 2, 308: 2, 311: 2, 312: 2, 300: 2, 314: 2, 270: 3, 354: 3, 306: 3, 309: 3, 305: 3, 315: 3, 296: 3, 299: 3, 271: 4, 288: 4, 345: 4, 425: 4, 3: 4, 4: 4, 301: 4, 310: 4, 1020: 4, 313: 4, 321: 4, 291: 4, 295: 4, 5: 4, 297: 4, 327: 5, 279: 5, 287: 5, 355: 5, 387: 5, 6: 5, 428: 5, 302: 5, 278: 5, 74: 5, 304: 5, 79: 5, 80: 5, 84: 5, 85: 5, 86: 5, 88: 5, 89: 5, 90: 5, 105: 5, 107: 5, 319: 5, 335: 5, 292: 5, 275: 5, 298: 5, 331: 6, 282: 6, 286: 6, 32: 6, 356: 6, 358: 6, 386: 6, 410: 6, 424: 6, 456: 6, 360: 6, 2: 6, 303: 6, 273: 6, 322: 6, 842: 6, 76: 6, 849: 6, 317: 6, 334: 6, 293: 6, 294: 6, 284: 6, 274: 6, 29: 7, 326: 7, 30: 7, 347: 7, 417: 7, 423: 7, 357: 7, 429: 7, 353: 7, 344: 7, 411: 7, 427: 7, 452: 7, 8: 7, 488: 7, 489: 7, 435: 7, 208: 7, 277: 7, 77: 7, 81: 7, 82: 7, 83: 7, 87: 7, 318: 7, 844: 7, 26: 7, 75: 7, 103: 7, 104: 7, 106: 7, 108: 7, 109: 7, 843: 7, 848: 7, 25: 7, 333: 7, 841: 7, 27: 7, 325: 7, 332: 7, 283: 7, 285: 7, 281: 7, 7: 7, 280

In [12]:
#set objective and open the network
SO = False # True - System optimum, False - User equilibrium
# barcelonaSubset = tncm.Network(link_file, trip_file, od_vols, origins, od_dic, links, graph)
barcelonaSubset = tncm.Network(link_file, trip_file, od_vols, origins, od_dic, links, graph)
#CM - start of initialization

# define output variables, network and fwResult
network = {(u,v): {'t0':d['object'].t0, 'alpha':d['object'].alpha, \
        'beta':d['object'].beta, 'capa':d['object'].capacity, 'flow':[], \
        'auxiliary':[], 'cost':[]} for (u, v, d) in barcelonaSubset.graph.edges(data=True)}

fwResult = {'theta':[], 'z':[]}

# initial all-or-nothing assignment and update link travel time(cost)
barcelonaSubset.all_or_nothing_assignment()
barcelonaSubset.update_linkcost()

for linkKey, linkVal in network.items():
    linkVal['cost'].append(barcelonaSubset.graph[linkKey[0]][linkKey[1]]['weight'])
    linkVal['auxiliary'].append(barcelonaSubset.graph[linkKey[0]][linkKey[1]]['object'].vol)
    linkVal['flow'].append(barcelonaSubset.graph[linkKey[0]][linkKey[1]]['object'].vol)


i 1


NodeNotFound: Node 1 not found in graph

In [11]:
SO=False
## iterations
iterNum=0
iteration = True
while iteration:
    iterNum += 1
    barcelonaSubset.all_or_nothing_assignment()
    barcelonaSubset.update_linkcost()
    
    # set auxiliary flow using updated link flow
    for linkKey, linkVal in network.items():
        linkVal['auxiliary'].append(barcelonaSubset.graph[linkKey[0]][linkKey[1]]['object'].vol)
        
    # getting optimal move size (theta)
    theta = lineSearch(network, SO)
    fwResult['theta'].append(theta)
    
    # set link flow (move) based on the theta, auxiliary flow, and link flow of previous iteration
    for linkKey, linkVal in network.items():
        aux = linkVal['auxiliary'][-1]
        flow = linkVal['flow'][-1]
        linkVal['flow'].append(flow + theta*(aux-flow))
        
        barcelonaSubset.graph[linkKey[0]][linkKey[1]]['object'].vol =  flow + theta * (aux - flow)
        barcelonaSubset.graph[linkKey[0]][linkKey[1]]['object'].flow = flow + theta * (aux - flow)
        
    # update link travel time
    barcelonaSubset.update_linkcost()
    
    # calculate objective function value
    z=0
    for linkKey, linkVal in network.items():
        linkVal['cost'].append(barcelonaSubset.graph[linkKey[0]][linkKey[1]]['weight'])
        totalcost = barcelonaSubset.graph[linkKey[0]][linkKey[1]]['object'].get_objective_function()
        z+=totalcost
        
    fwResult['z'].append(z)        
        
    # convergence test
    if iterNum == 1:
        iteration = True
    else:
        if abs(fwResult['z'][-2] - fwResult['z'][-1]) <= 0.001 or iterNum==3000:
            iteration = False


i 1


NodeNotFound: Node 1 not found in graph